# **Tugas : Clustering**
1. Data : Boston House Proce (Gunakan hanya feature numerikal)
2. Perlakuan : Tentukan harus pake scaling atau tidak
3. Gunakan K Means Clustering, tentukan best K nya.
4. Buatlah visualisasi setelah mendapatkan clusternya. Coba lihat persebaran clusternya secara umum (bisa dengan 2 atau 3 dimensi)
5. Gambarkan secara umum setiap clusternya (berikan label)

In [214]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from scipy.spatial.distance import cdist,pdist
from sklearn.metrics import silhouette_score

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA

# impute missing values
from sklearn.impute import SimpleImputer # mean, median, most_frequent (mode), constant
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer # regresi
from sklearn.impute import KNNImputer # regresi KKN

# scaling
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler 

# column transformer & pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')

In [215]:
headers = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE',
           'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'PRICE']
df = pd.read_csv('housing.csv', sep='\s+', names= headers)
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [216]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    int64  
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    int64  
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  PRICE    506 non-null    float64
dtypes: float64(12), int64(2)
memory usage: 55.5 KB


In [217]:
df.isna().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
PRICE      0
dtype: int64

In [218]:
# dfTemp = []
print(f'Column\t| lowerBound \t| upperBound\n===========================================')
for i in df:
    q1 = df[i].describe()['25%']
    q3 = df[i].describe()['75%']
    IQR = q3-q1
    lowerBound = q1 - (3 * IQR)
    upperBound = q3 + (3 * IQR)
    print(f'{i}\t|{lowerBound}\t|{upperBound}')
    df = df[df[i].between(lowerBound,upperBound)]

Column	| lowerBound 	| upperBound
CRIM	|-10.7030675	|14.462195000000001
ZN	|-60.0	|80.0
INDUS	|-33.540000000000006	|56.830000000000005
CHAS	|0.0	|0.0
NOX	|-0.04700000000000004	|1.10975
RM	|3.830750000000002	|8.644999999999998
AGE	|-108.7	|244.8
DIS	|-6.835900000000001	|14.379700000000001
RAD	|-8.0	|20.0
TAX	|-72.0	|726.0
PTRATIO	|7.600000000000001	|28.6
B	|342.70000000000016	|437.54999999999984
LSTAT	|-13.659999999999997	|33.012499999999996
PRICE	|-5.5000000000000036	|52.6


In [219]:
len(df)

311

In [220]:
q1 = df['CRIM'].describe()['25%']
display(q1)
q3 = df['CRIM'].describe()['75%']
display(q3)
IQR = q3-q1
display(IQR)
lowerBound = q1 - (3 * IQR)
display(lowerBound)
upperBound = q3 + (3 * IQR)
display(upperBound)
#df['CRIM'] = df['CRIM'].between(lowerBound,upperBound)

0.061135

0.294545

0.23341

-0.639095

0.994775

In [221]:
#len(df[(df['CRIM'] > upperBound) | (df['CRIM'] < lowerBound)])
#df[(df['CRIM'] <= upperBound) & (df['CRIM'] >= lowerBound)]
df['CRIM'] = df[df['CRIM'].between(lowerBound,upperBound)]['CRIM']
#df[df['CRIM'].between(lowerBound,upperBound)]
#df[df['CRIM'] < lowerBound]

In [222]:
df['CRIM']

0      0.00632
1      0.02731
2      0.02729
3      0.03237
4      0.06905
        ...   
501    0.06263
502    0.04527
503    0.06076
504    0.10959
505    0.04741
Name: CRIM, Length: 311, dtype: float64

# PCA

In [223]:
# Instantiate the PCA object
pca = PCA(n_components=3)  # Number of components to keep

# Fit the PCA model to the data
pca.fit(df)

# Access the explained variance ratio of each component
explained_variance_ratio = pca.explained_variance_ratio_

# Access the principal components
principal_components = pca.components_

# Transform the data to the principal component space
X_transformed = pca.transform(df)

X_transformed

ValueError: Input X contains NaN.
PCA does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
df.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'PRICE'],
      dtype='object')

In [ ]:
transformer = ColumnTransformer(
    [
        ('RobustScale',RobustScaler(),['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'PRICE'])
    ],
    remainder='passthrough' #LEWATKAN YANG GAK DIMENTION
)

In [ ]:
df_scaled = transformer.fit_transform(df)
df_scaled = pd.DataFrame(df_scaled, columns=headers)

In [ ]:
df_scaled

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,-0.069593,1.44,-0.571650,0.0,0.000000,0.496612,-0.250765,0.285777,-0.20,-0.087855,-1.339286,0.261902,-0.637681,0.351097
1,-0.063755,0.00,-0.202943,0.0,-0.394286,0.287940,0.028542,0.569789,-0.15,-0.227390,-0.446429,0.261902,-0.221889,0.050157
2,-0.063760,0.00,-0.202943,0.0,-0.394286,1.323171,-0.334353,0.569789,-0.15,-0.227390,-0.446429,0.066675,-0.732634,1.692790
3,-0.062347,0.00,-0.581720,0.0,-0.457143,1.069783,-0.646279,0.924391,-0.10,-0.279070,-0.125000,0.153016,-0.841579,1.529781
4,-0.052144,0.00,-0.581720,0.0,-0.457143,1.271680,-0.475025,0.924391,-0.10,-0.279070,-0.125000,0.261902,-0.602699,1.880878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,-0.053930,0.00,0.173509,0.0,0.200000,0.521003,-0.171254,-0.236007,-0.20,-0.147287,0.696429,0.026382,-0.168916,0.150470
502,-0.058759,0.00,0.173509,0.0,0.200000,-0.119919,-0.016310,-0.297887,-0.20,-0.147287,0.696429,0.261902,-0.227886,-0.075235
503,-0.054450,0.00,0.173509,0.0,0.200000,1.039973,0.275229,-0.336744,-0.20,-0.147287,0.696429,0.261902,-0.571714,0.338558
504,-0.040867,0.00,0.173509,0.0,0.200000,0.793360,0.240571,-0.265053,-0.20,-0.147287,0.696429,0.096414,-0.487756,0.100313


In [ ]:
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0
